In [1]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [2]:
def parse_scans(board,voltage,datecut):
    def get_phase_from_time(fname):
        try:
            x=np.loadtxt(fname,delimiter=',',dtype=int)
        except:
            x=np.ones(15*12,dtype=int).reshape(15,12)*999
        return x
    def get_best_setting(err_counts):
        best_setting_by_ch = []
        counts_window = []
        for i in range(15):
            counts_window.append( err_counts[i] + err_counts[(i-1)%15] + err_counts[(i+1)%15])
        counts_window = np.array(counts_window)
        counts_window[ err_counts>0 ] += 255*3
        y = (err_counts[2:-2]+err_counts[1:-3]+err_counts[3:-1]+err_counts[4:] + err_counts[:-4])
        y[ err_counts[2:-2]>0 ] += 2555
        best_setting = y.argmin(axis=0)+2
        for ch in range(12):
            best_setting_by_ch.append(best_setting[ch])
        return np.array(best_setting_by_ch)

    directory = f"../../PhaseScans/board_{board}/voltage_{voltage}"
    scanDirs = os.listdir(directory)
    
    df=pd.DataFrame({'Timestamp':scanDirs})
    df['PLLscan_time']=df['Timestamp'].apply(lambda x: pd.to_datetime(x,format="%d%b_%H%M%S").replace(year=2022))
    df=df.loc[df.PLLscan_time>'2022-09-26 12']
    df.set_index('PLLscan_time',inplace=True)
    df.sort_index(inplace=True)

    allowed_capSel = np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,                                                                                                                                             
                                  13,  14,  15,  24,  25,  26,  27,  28,  29,  30,  31,  56,  57,                                                                                                                                            
                                  58,  59,  60,  61,  62,  63, 120, 121, 122, 123, 124, 125, 126,                                                                                                                                            
                                  127, 248, 249, 250, 251, 252, 253, 254, 255, 504, 505, 506, 507,                                                                                                                                           
                                  508, 509, 510, 511]) 
    
    for i,capSel in enumerate(allowed_capSel):
        df[f'phaseScan_1p20V_CapSel{capSel}']=df['Timestamp'].apply(lambda x: get_phase_from_time(f'../../PhaseScans/board_10/voltage_1_20/{x}/eTx_DelayScan_CapSelect_{capSel}.csv'))
        df[f'phaseScan_1p08V_CapSel{capSel}']=df['Timestamp'].apply(lambda x: get_phase_from_time(f'../../PhaseScans/board_10/voltage_1_08/{x}/eTx_DelayScan_CapSelect_{capSel}.csv'))
        df[f'phaseScan_1p32V_CapSel{capSel}']=df['Timestamp'].apply(lambda x: get_phase_from_time(f'../../PhaseScans/board_10/voltage_1_32/{x}/eTx_DelayScan_CapSelect_{capSel}.csv'))
    
        df_cap = df.copy()
        df_cap['CapSel'] = capSel
        df_cap['phase_scan'] = df_cap['Timestamp'].apply(lambda x: get_phase_from_time(f'{directory}/{x}/eRx_PhaseScan_CapSelect_{capSel}.csv'))
        #err_counts = df_cap['phase_scan'].values
        df_cap['best_phase'] = df_cap['phase_scan'].apply(lambda x: get_best_setting(x))
        if i==0:
            df_new = df_cap.copy()
        else:
            df_new = pd.concat([df_cap, df_new])

                
    df_new = df_new.reset_index()
    return df_new

In [3]:
def parse_pll(fname):
    _f = open(f'../../logs/{fname}')
    _fLines = _f.read()
    def find_match(re_string):
        return np.array(re.findall(re_string,_fLines))
    
    output = {}

    # cond_matches = find_match(r'09-(.*) INFO   Power: On, Voltage: (.*) V, Current: (.*) A, Temp: (.*) C, Res.: (.*) Ohms\n09.* Word count (\d*), error count (\d*)')
    # dates = pd.to_datetime(['09-'+x for x in cond_matches[:,0]])
    
    pll_set_matches = find_match("09-(.*) INFO   Good PLL settings V=(1.20):|09-(.*) INFO.     CapSel=(\d+), V=(1.20), PUSM=(\d)")
    pll_good_matches = find_match('09-(.*) INFO   Setting PLL VCO CapSelect to (\d*) with phaseSelect settings of (\d*),(\d*),(\d*),(\d*),(\d*),(\d*),(\d*),(\d*),(\d*),(\d*),(\d*),(\d*)')
    pll_timestamp_matches = find_match('09-(.*) INFO   Starting Power Scans \( timestamp (.*) \)')
    pll_scan_time = pd.to_datetime(['09-'+x for x in pll_good_matches[:,0]])
    pll_voltage = pll_set_matches[:,1][pll_set_matches[:,1] == '1.20']
    pll_time = pd.to_datetime(['09-'+x for x in pll_set_matches[:,0][pll_set_matches[:,1] == '1.20']])
    pll_ones = np.ones_like(pll_voltage,dtype=int)
    
    output['PLLscan_time'] = pll_scan_time
    output['PLLscan_voltage'] = np.float32(pll_voltage)
    output['PLLscan_choice'] = pll_good_matches[:,1]
    # need to filter timestamps within  12:34:31 INFO   Setting PLL VCO CapSelect 
    time_ts =  pd.to_datetime(['09-'+x for x in pll_timestamp_matches[:,0]])
    print(time_ts)
    print(pll_time)
    print(pll_scan_time)
    output['PLLscan_timestamp'] = pll_timestamp_matches[:,1]
    output['PLLscan_bestPhase'] = pll_good_matches[:,2:].tolist()
    
    df = pd.DataFrame.from_dict(output)
    #df['PLLscan_bestPhase'] = pll_good_matches[:,2:].tolist()

    return  df

In [4]:
voltage = "1_20"
voltages = {"1_20":1.2,"1_32":1.32,"1.08":1.08}
log_strs = [
    'logFile_Chip10_CoolDown.log',
    
    #'logFile_Chip10_StartIrradiation.log'
]
df_irr = pd.concat([parse_pll(lname) for lname in log_strs])
df_scans = parse_scans('10','1_20','2022-09-26 12')

DatetimeIndex(['2022-09-26 12:02:57', '2022-09-26 12:29:35',
               '2022-09-26 12:57:55', '2022-09-26 13:25:37',
               '2022-09-26 13:52:47', '2022-09-26 14:20:38'],
              dtype='datetime64[ns]', freq=None)
DatetimeIndex(['2022-09-26 12:06:39', '2022-09-26 12:33:06',
               '2022-09-26 13:01:36', '2022-09-26 13:29:14',
               '2022-09-26 13:56:28', '2022-09-26 14:24:15'],
              dtype='datetime64[ns]', freq=None)
DatetimeIndex(['2022-09-26 12:08:09', '2022-09-26 12:34:31',
               '2022-09-26 13:02:56', '2022-09-26 13:30:39',
               '2022-09-26 13:57:54', '2022-09-26 14:25:40'],
              dtype='datetime64[ns]', freq=None)


/Users/cmantill/miniconda3/envs/econ-sw/lib/python3.7/site-packages/ipykernel_launcher.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/Users/cmantill/miniconda3/envs/econ-sw/lib/python3.7/site-packages/ipykernel_launcher.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/Users/cmantill/miniconda3/envs/econ-sw/lib/python3.7/site-packages/ipykernel_launcher.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

In [5]:
df_irr.columns

Index(['PLLscan_time', 'PLLscan_voltage', 'PLLscan_choice',
       'PLLscan_timestamp', 'PLLscan_bestPhase'],
      dtype='object')

In [6]:
df_scans.columns

Index(['PLLscan_time', 'Timestamp', 'phaseScan_1p20V_CapSel0',
       'phaseScan_1p08V_CapSel0', 'phaseScan_1p32V_CapSel0',
       'phaseScan_1p20V_CapSel1', 'phaseScan_1p08V_CapSel1',
       'phaseScan_1p32V_CapSel1', 'phaseScan_1p20V_CapSel2',
       'phaseScan_1p08V_CapSel2',
       ...
       'phaseScan_1p32V_CapSel509', 'phaseScan_1p20V_CapSel510',
       'phaseScan_1p08V_CapSel510', 'phaseScan_1p32V_CapSel510',
       'phaseScan_1p20V_CapSel511', 'phaseScan_1p08V_CapSel511',
       'phaseScan_1p32V_CapSel511', 'CapSel', 'phase_scan', 'best_phase'],
      dtype='object', length=173)

In [7]:
# 1. plot the phaseSelect diagram, for the PLL setting that was actually used for that region
timecut_irr = df_irr.loc[(df_irr.PLLscan_time<'2022-09-26 12') & (df_irr.PLLscan_time>'2022-09-26 12:34')]
capsel = int(timecut_irr['PLLscan_choice'].values[0])
bestphase = np.array([int(p) for p in timecut_irr['PLLscan_bestPhase'].values[0]])
print(bestphase)

timecut_scans = df_scans.loc[(df_scans.PLLscan_time<'2022-09-27 23') & (df_scans.PLLscan_time>'2022-09-27 22:30')]
phasescan = timecut_scans.loc[timecut_scans.CapSel == capsel]['phase_scan'].values
bestphase_fromscan = timecut_scans.loc[timecut_scans.CapSel == capsel]['best_phase'].values
print(phasescan)
print(bestphase_fromscan)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
df_scans.PLLscan_timestamp

In [ ]:
df_irr.PLLscan_voltage